In [ ]:
import os

import cv2
import numpy as np
import tensorflow as tf
from huggingface_hub import from_pretrained_keras

IMG_SIZE = 224

In [ ]:
backbone = from_pretrained_keras("keras-io/monocular-depth-estimation")

In [ ]:
base_path = "/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/"
live_path = os.path.join(base_path, "4930/live/504311.png")
bbox_live = "".join(live_path.split(".")[:-1]) + "_BB.txt"
spoof_path = os.path.join(base_path, "3613/spoof/541354.png")
bbox_spoof = "".join(spoof_path.split(".")[:-1]) + "_BB.txt"

In [ ]:
# Load the images
live_image = cv2.imread(live_path)
spoof_image = cv2.imread(spoof_path)

# Extract bounding boxes
with open(bbox_live, "r") as file:
    bbox_live = file.readlines()[0]
    bbox_live = list(map(int, bbox_live.split()[:-1]))

with open(bbox_spoof, "r") as file:
    bbox_spoof = file.readlines()[0]
    bbox_spoof = list(map(int, bbox_spoof.split()[:-1]))


In [ ]:
# Show loaded images
cv2.imshow("Live Image", live_image)
cv2.imshow("Spoof Image", spoof_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
live_image = cv2.resize(live_image, (IMG_SIZE, IMG_SIZE))
spoof_image = cv2.resize(spoof_image, (IMG_SIZE, IMG_SIZE))

# Crop faces and show cropped images
live_image = live_image[
    bbox_live[1] : bbox_live[1] + bbox_live[3],
    bbox_live[0] : bbox_live[0] + bbox_live[2],
]
spoof_image = spoof_image[
    bbox_spoof[1] : bbox_spoof[1] + bbox_spoof[3],
    bbox_spoof[0] : bbox_spoof[0] + bbox_spoof[2],
]


In [ ]:
# Show cropped images

live_image = live_image.astype(np.float32)
spoof_image = spoof_image.astype(np.float32)
live_image = (live_image - np.mean(live_image)) / np.std(live_image)
spoof_image = (spoof_image - np.mean(spoof_image)) / np.std(spoof_image)
live_image = (live_image * 255).astype(np.uint8)
spoof_image = (spoof_image * 255).astype(np.uint8)
cv2.imshow("Live Image", live_image)
cv2.imshow("Spoof Image", spoof_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Preprocess the cropped images
live_image = cv2.resize(live_image, (IMG_SIZE, IMG_SIZE))
spoof_image = cv2.resize(spoof_image, (IMG_SIZE, IMG_SIZE))
cv2.imwrite("Live_Image.png", live_image)
cv2.imwrite("Spoof_Image.png", spoof_image)
live_image = cv2.cvtColor(live_image, cv2.COLOR_BGR2RGB)
spoof_image = cv2.cvtColor(spoof_image, cv2.COLOR_BGR2RGB)
# live_image = tf.image.convert_image_dtype(live_image, tf.float32)
# spoof_image = tf.image.convert_image_dtype(spoof_image, tf.float32)

live_image = live_image.astype(np.float32)
spoof_image = spoof_image.astype(np.float32)

# live_image = (
#     (live_image - np.min(live_image))
#     / (np.max(live_image) - np.min(live_image))
# )
# spoof_image = (
#     (spoof_image - np.min(spoof_image))
#     / (np.max(spoof_image) - np.min(spoof_image))
# )

live_image /= 127.5
live_image -= 1.0
spoof_image /= 127.5
spoof_image -= 1.0

# # z-score normalization + imagenet normalization + scaling to [-1, 1]

# live_image = (live_image - np.mean(live_image)) / np.std(live_image)
# spoof_image = (spoof_image - np.mean(spoof_image)) / np.std(spoof_image)

# live_image = (
#     (live_image - np.min(live_image))
#     / (np.max(live_image) - np.min(live_image))
# )
# spoof_image = (
#     (spoof_image - np.min(spoof_image))
#     / (np.max(spoof_image) - np.min(spoof_image))
# )
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]
# live_image = (live_image - mean) / std
# spoof_image = (spoof_image - mean) / std

# live_image = live_image * 2.0 - 1.0
# spoof_image = spoof_image * 2.0 - 1.0

# print(np.max(live_image))
# print(np.min(live_image))
# print(np.max(spoof_image))
# print(np.min(spoof_image))


live_image = tf.expand_dims(live_image, axis=0)
spoof_image = tf.expand_dims(spoof_image, axis=0)


# Make predictions
live_depth_map = backbone.predict(live_image)[0]
spoof_depth_map = backbone.predict(spoof_image)[0]



# Normalize the depth maps
live_depth_map = (
    (live_depth_map - np.min(live_depth_map))
    / (np.max(live_depth_map) - np.min(live_depth_map))
    * 255.0
)
spoof_depth_map = (
    (spoof_depth_map - np.min(spoof_depth_map))
    / (np.max(spoof_depth_map) - np.min(spoof_depth_map))
    * 255.0
)

# Convert the masks to uint8
live_depth_map = live_depth_map.astype(np.uint8)
spoof_depth_map = spoof_depth_map.astype(np.uint8)

cv2.imwrite("Live_Depth_Map.png", live_depth_map)
cv2.imwrite("Spoof_Depth_Map.png", spoof_depth_map)

# Show the predicted depth maps
cv2.imshow("Live Depth Map", live_depth_map)
cv2.imshow("Spoof Depth Map", spoof_depth_map)
cv2.waitKey(0)
cv2.destroyAllWindows()